In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class AutoEncoder(nn.Module):
    def __init__(self):

        super().__init__()
        self.encoder = nn.Sequential( # input : 3601*3601
            nn.Conv2d(1, 32, kernel_size=(15, 15), stride=4, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(32, 64, kernel_size=(8, 8), stride=4, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(64, 128, kernel_size=(4, 4), stride=4, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(128, 256, kernel_size=(4, 4), stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(256, 512, kernel_size=(4, 4), stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(512, 1024, kernel_size=(4, 4), stride=2, padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(1024, 2048, kernel_size=(4, 4), stride=2, padding=1),
            nn.BatchNorm2d(2048),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Flatten(),
            nn.Linear(18432, 5000),
            nn.ReLU(),
            nn.Linear(5000, 1000),
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.Linear(1000, 5000),
            nn.ReLU(),
            nn.Linear(5000, 18432),
            nn.ReLU(),
            nn.Unflatten(1, (2048, 3, 3)),
            nn.ConvTranspose2d(2048, 1024, kernel_size=(4, 4), stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=2, padding=1, output_padding=0),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=2, padding=1, output_padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=2, padding=1, output_padding=0),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=4, padding=1, output_padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(64, 32, kernel_size=(8, 8), stride=4, padding=1, output_padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(32, 1, kernel_size=(15, 15), stride=4, padding=1, output_padding=0),
            nn.Sigmoid()

        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def print_shapes(self, x):
      print(f"----------ENCODER----------")
      print(f"Input shape : {x.shape}")

      for i, layer in enumerate(self.encoder):
          x = layer(x)
          print(f"After layer {i+1} ({type(layer).__name__}): {x.shape}")

      print(f"----------DECODER----------")

      for i, layer in enumerate(self.decoder):
          x = layer(x)
          print(f"After layer {i+1} ({type(layer).__name__}): {x.shape}")


In [4]:
load = torch.load(r'/content/drive/My Drive/Data/DEM_elevation_tensor.pt')

In [5]:
for key in load:
    matrix_min = load[key].min()
    matrix_max = load[key].max()
    load[key] = (load[key] - matrix_min) / (matrix_max - matrix_min)

In [6]:
class DEM_Dataset(Dataset):
    def __init__(self, data):
        self.filename = list(data.keys())
        self.data = data

    def __len__(self):
        return len(self.filename)

    def __getitem__(self, index):
        filename = self.filename[index]
        data = self.data[filename]
        return data

In [7]:
model = AutoEncoder()

In [8]:
dummy_input = torch.randn(4, 1, 3601, 3601)
model.dimensions(dummy_input)

In [9]:
data = DEM_Dataset(load)
batch_size = 4
train_size = 35
test_size = 7
val_size = 8

train_dataset, test_dataset, val_dataset = random_split(data, [train_size, test_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [10]:
model = AutoEncoder()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


AutoEncoder(
  (encoder): Sequential(
    (0): Conv2d(1, 32, kernel_size=(15, 15), stride=(4, 4), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Conv2d(32, 64, kernel_size=(8, 8), stride=(4, 4), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Conv2d(64, 128, kernel_size=(4, 4), stride=(4, 4), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.2, inplace=False)
    (12): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Dropout(p=0.2, inplace=False)
    (16): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 

In [11]:
from copy import deepcopy

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
num_epochs = 100
best_model = None
best_loss = float('inf')
patience = 100
patience_counter = 0


for epoch in tqdm(range(num_epochs), desc='Epochs'):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        batch_data = batch_data.float().to(device)
        optimizer.zero_grad()
        #print(batch_data.unsqueeze(1).shape)
        #model.dimensions(batch_data.unsqueeze(1))
        outputs = model(batch_data.unsqueeze(1))
        loss = criterion(outputs, batch_data.unsqueeze(1))
        loss.backward()
        optimizer.step()
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():
        for val_batch_data in val_loader:
            val_batch_data = val_batch_data.float().to(device)
            val_outputs = model(val_batch_data.unsqueeze(1))
            val_loss += criterion(val_outputs, val_batch_data.unsqueeze(1)).item()

    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss}")

     # Check if the validation loss has improved
    if val_loss < best_loss:
        best_loss = val_loss
        best_model = deepcopy(model)  # Save a copy of the model
        patience_counter = 0  # Reset patience counter
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping triggered. Stopped at epoch {epoch+1}. Best epoch was {epoch+1-patience_counter} with val loss: {best_loss}")
        break

Epochs:   1%|          | 1/100 [00:10<17:49, 10.80s/it]

Epoch 1/100, Loss: 0.07033410668373108, Val Loss: 0.048449842259287834


Epochs:   2%|▏         | 2/100 [00:12<09:17,  5.69s/it]

Epoch 2/100, Loss: 0.0770944133400917, Val Loss: 0.04840477369725704


Epochs:   3%|▎         | 3/100 [00:14<06:32,  4.04s/it]

...

Epoch 98/100, Loss: 0.028926441445946693, Val Loss: 0.03493987116962671


Epochs:  99%|█████████▉| 99/100 [03:36<00:02,  2.10s/it]

Epoch 99/100, Loss: 0.047732166945934296, Val Loss: 0.0364910289645195


Epochs: 100%|██████████| 100/100 [03:38<00:00,  2.19s/it]

Epoch 100/100, Loss: 0.013323843479156494, Val Loss: 0.029318821616470814


In [12]:
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
with torch.no_grad():
    with tqdm(test_loader, desc='Testing', leave=False) as t:
        for test_batch_data in t:
            test_batch_data = test_batch_data.float().to(device)
            test_outputs = model(test_batch_data.unsqueeze(1))
            test_loss += criterion(test_outputs, test_batch_data.unsqueeze(1)).item()

test_loss /= len(test_loader)  # Average the test loss
print(f"Test Loss: {test_loss}")

Test Loss: 0.047876796685159206


In [13]:
#torch.save(embedded_dict, "/home/smitesh22/Unsupervised-Machine-Learning-for-Solar-Site-Selection/Code/embedded_dict.pt")

torch.save(model, "/content/drive/My Drive/Thesis/Model/EncoderDecoderElevationModel.pt")